**Srishti** **Sharma**

---



In [1]:
!pip install mne -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 99.4 MB/s eta 0:00:00


MNE (Magnetoencephalography and Electroencephalography) is a Python package for neuroimaging data analysis. It provides a wide range of functions for preprocessing, analyzing, and visualizing electrophysiological data, such as EEG (electroencephalography), MEG (magnetoencephalography), ECoG (electrocorticography), and SEEG (stereotactic electroencephalography).

**Time-frequency analysis:** MNE provides methods for analyzing oscillatory activity in the time-frequency domain, such as the Morlet wavelet transform, multitaper spectral analysis, and Welch's periodogram method.

* https://www.biorxiv.org/content/10.1101/324954v1  I've taken this research paper for reference.
* Datasets were collected using EMOTIVE with the 128 Hz sampling frequency.   
* There are two differnt dataset collected in Guinea Bissau (97 subjects) and Nigeria (112 subjects).  


In [2]:
!wget https://zenodo.org/record/1252141/files/EEGs_Guinea-Bissau.zip

--2023-05-01 21:47:15--  https://zenodo.org/record/1252141/files/EEGs_Guinea-Bissau.zip
Resolving zenodo.org (zenodo.org)... 188.185.124.72
Connecting to zenodo.org (zenodo.org)|188.185.124.72|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 153973086 (147M) [application/octet-stream]
Saving to: ‘EEGs_Guinea-Bissau.zip’

EEGs_Guinea-Bissau. 100%[===================>] 146.84M  28.1MB/s    in 6.1s    

2023-05-01 21:47:22 (23.9 MB/s) - ‘EEGs_Guinea-Bissau.zip’ saved [153973086/153973086]



In [3]:
!wget https://zenodo.org/record/1252141/files/EEGs_Nigeria.zip

--2023-05-01 21:47:57--  https://zenodo.org/record/1252141/files/EEGs_Nigeria.zip
Resolving zenodo.org (zenodo.org)... 188.185.124.72
Connecting to zenodo.org (zenodo.org)|188.185.124.72|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 276402063 (264M) [application/octet-stream]
Saving to: ‘EEGs_Nigeria.zip’

EEGs_Nigeria.zip    100%[===================>] 263.60M  29.5MB/s    in 9.6s    

2023-05-01 21:48:08 (27.5 MB/s) - ‘EEGs_Nigeria.zip’ saved [276402063/276402063]



# Unzip the data

In [4]:
#unzip the files
from zipfile import ZipFile 
data = ZipFile('EEGs_Guinea-Bissau.zip')
data.extractall()

In [5]:
#unzip the files
from zipfile import ZipFile 
data_n = ZipFile('EEGs_Nigeria.zip')
data_n.extractall()

# Read data

In [20]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import sklearn

In [11]:
meta_n=pd.read_csv('https://zenodo.org/record/1252141/files/metadata_nigeria.csv')
meta_n.head()

,subject.id,recordedPeriod,startTime,session.id,first_condition,remarks,Group,csv.file
0,6,270,26/9/2016 13:13,1,open,NaN,control,signal-6-1.csv.gz
1,9,271,26/9/2016 13:30,1,closed,NaN,control,signal-9-1.csv.gz
2,10,272,26/9/2016 13:36,1,open,eyes closed at 2:40,control,signal-10-1.csv.gz
3,11,274,26/9/2016 13:42,2,closed,no.11.1 failed >> 11.2 is the right one,control,signal-11-2.csv.gz
4,11,1,26/9/2016 13:42,1,closed,no.11.1 failed >> 11.2 is the right one,control,signal-11-1.csv.gz


In [12]:
meta_df=pd.read_csv('https://zenodo.org/record/1252141/files/metadata_guineabissau.csv')
meta_df.head()

,subject.id,Group,Eyes.condition,Remarks,recordedPeriod,startTime
0,1,Epilepsy,closed-3min-then-open-2min,by 45s reposition electrodes,301,27/5/2020 14:33
1,2,Control,open-3min-then-closed-2min,NaN,309,26/5/2020 22:44
2,3,Epilepsy,closed-3min-then-open-2min,NaN,309,27/5/2020 14:26
3,4,Epilepsy,closed-3min-then-open-2min,"Green lights not shown, but good EEG traces",299,27/5/2020 15:23
4,5,Control,closed-3min-then-open-2min,NaN,302,23/5/2020 19:09


In [13]:
#now i need to seprate Epilepsy vs Control subjects
EP_sub=meta_df['subject.id'][meta_df['Group']=='Epilepsy']
CT_sub=meta_df['subject.id'][meta_df['Group']=='Control']



In [14]:
#reading csv files
Epilepsy=[pd.read_csv('/content/EEGs_Guinea-Bissau/signal-1.csv.gz'.format(i), compression='gzip') for i in  EP_sub]
Control=[pd.read_csv('/content/EEGs_Guinea-Bissau/signal-1.csv.gz'.format(i), compression='gzip') for i in  CT_sub]

In [ ]:
Epilepsy[0].head()

,Unnamed: 0,AF3,AF4,F3,F4,F7,F8,FC5,FC6,O1,...,CQ_F3,CQ_P7,CQ_P8,CQ_F4,CQ_AF3,CQ_FC5,CQ_O1,CQ_T8,CQ_F8,CQ_DRL
0,1,4426.153846,3994.871795,4408.205128,3847.692308,4690.256410,3895.897436,4702.051282,3914.871795,4049.743590,...,4,4,4,4,4,4,4,4,4,4
1,2,4420.512821,3986.666667,4394.358974,3836.923077,4678.461538,3886.666667,4696.410256,3910.769231,4054.358974,...,4,4,4,4,4,4,4,4,4,4
2,3,4413.846154,3986.153846,4386.666667,3831.794872,4654.871795,3881.025641,4690.769231,3908.205128,4066.666667,...,4,4,4,4,4,4,4,4,4,4
3,4,4407.692308,3984.615385,4384.102564,3832.820513,4644.615385,3883.076923,4686.153846,3910.256410,4063.076923,...,4,4,4,4,4,4,4,4,4,4
4,5,4407.179487,3978.974359,4382.564103,3832.307692,4647.692308,3878.974359,4685.641026,3903.076923,4057.948718,...,4,4,4,4,4,4,4,4,4,4


In [15]:
#remove non eeg channels
Epilepsy=[i.iloc[:,1:15] for i in  Epilepsy]
Control=[i.iloc[:,1:15] for i in  Control]

# Convert to MNE object



1.**info=mne.create_info(list(sub.columns), ch_types=['eeg'] * len(sub.columns), sfreq=128):** This line creates an Info object, which stores information about the data, such as channel names, channel types, and sampling rate. The channel names are taken from the column names of the input DataFrame sub, and the channel types are set to 'eeg'. The sampling rate is set to 128 Hz.

2.**info.set_montage('standard_1020'):** This line sets the electrode positions using the standard 10-20 system.

3.**data=mne.io.RawArray(sub.T, info):** This line creates a RawArray object, which stores the raw EEG data as a 2D array (channels x samples). The input DataFrame sub is transposed (i.e., rows become columns and columns become rows) so that the channels are in the correct orientation for the RawArray.

4.**data.set_eeg_reference():** This line applies an average reference to the EEG data.

5.**data.filter(l_freq=0.1,h_freq=45):** This line applies a band-pass filter to the data, with a lower cutoff frequency of 0.1 Hz and an upper cutoff frequency of 45 Hz.

6.**epochs=mne.make_fixed_length_epochs(data,duration=5,overlap=1):** This line creates fixed-length epochs from the data, with each epoch lasting 5 seconds and overlapping with the previous epoch by 1 second.

7.**epochs=epochs.drop_bad():** This line removes epochs that contain artifacts or other bad data.

8.**return epochs:** The function returns the Epochs object.

Overall, this function takes a DataFrame of EEG data, sets the channel names, types, and positions, applies an average reference and band-pass filter, creates fixed-length epochs, and removes bad epochs. The resulting Epochs object can then be used for further analysis, such as computing event-related potentials or time-frequency decompositions.

In [16]:
import mne
def convertDF2MNE(sub):
    info = mne.create_info(list(sub.columns), ch_types=['eeg'] * len(sub.columns), sfreq=128)
    info.set_montage('standard_1020')
    data=mne.io.RawArray(sub.T, info)
    data.set_eeg_reference()
    data.filter(l_freq=0.1,h_freq=45)
    epochs=mne.make_fixed_length_epochs(data,duration=5,overlap=1)
    epochs=epochs.drop_bad()
    
    return epochs

The %%capture magic command is used to suppress the output of the function calls, which can be quite verbose in MNE.

In [17]:
%%capture 
#Convert each dataframe to mne object
Epilepsy=[convertDF2MNE(i) for i in  Epilepsy]
Control=[convertDF2MNE(i) for i in  Control]

In [ ]:
%%capture
Epilepsy_epochs=mne.concatenate_epochs(Epilepsy)
Control_epochs=mne.concatenate_epochs(Control)

# This code cell concatenates the Epochs objects in the Epilepsy and Control lists into single Epochs objects
# using the mne.concatenate_epochs function.
# The resulting Epilepsy_epochs and Control_epochs objects contain all of the EEG data from 
# the respective groups, and can be used for further analysis.

# Create labels and groups

In [ ]:
Epilepsy_group=np.concatenate([[i]*len(Epilepsy[i]) for i in range(len(Epilepsy))])#create a list of list where each sub list corresponds to subject_no
Control_group=np.concatenate([[i]*len(Control[i]) for i in range(len(Control))])#create a list of list where each sub list corresponds to subject_no

Epilepsy_label=np.concatenate([[0]*len(Epilepsy[i]) for i in range(len(Epilepsy))])
Control_label=np.concatenate([[1]*len(Control[i]) for i in range(len(Control))])

In [ ]:
Epilepsy_group.shape,Control_group.shape,Epilepsy_label.shape,Control_label.shape

((3995,), (3461,), (3995,), (3461,))

In [ ]:
#combine data
data=mne.concatenate_epochs([Epilepsy_epochs,Control_epochs])
group=np.concatenate((Epilepsy_group,Control_group))
label=np.concatenate((Epilepsy_label,Control_label))
print(len(data),len(group),len(label))

Not setting metadata
7456 matching events found
No baseline correction applied
7456 7456 7456


# Feature Extraction - Power spectral density
The power spectral density of a signal is a measure of how much power the signal has at each different frequency. The power spectral density (power spectrum) reflects the ‘frequency content’ of the signal or the distribution of signal power over frequency.  


Time-series signal can be converted to frequency domain via various methods


1. Welch power density method
2. Fast Fourier transform (FFT)

In [ ]:

from mne.time_frequency import psd_welch
def eeg_power_band(epochs):

    # This function takes an ``mne.Epochs`` object and creates EEG features based
    # on relative power in specific frequency bands that are compatible with
    # scikit-learn.

    
    # specific frequency bands
    FREQ_BANDS = {"delta": [0.5, 4.5],
                  "theta": [4.5, 8.5],
                  "alpha": [8.5, 11.5],
                  "sigma": [11.5, 15.5],
                  "beta": [15.5, 30],
                  "gamma": [30, 45],
                  }

    psds, freqs = psd_welch(epochs, picks='eeg', fmin=0.5, fmax=45)# Compute the PSD using the Welch method
    psds /= np.sum(psds, axis=-1, keepdims=True)    # Normalize the PSDs

    X = []#For each frequency band, compute the mean PSD in that band
    for fmin, fmax in FREQ_BANDS.values():
        psds_band = psds[:, :, (freqs >= fmin) & (freqs < fmax)].mean(axis=-1)# Compute the mean PSD in each frequency band.
        X.append(psds_band)

    return np.concatenate(X, axis=1)#Concatenate the mean PSDs for each band into a single feature vector

'eeg_power_band' function, extracts EEG features based on the relative power in specific frequency bands using the psd_welch function from the MNE package.

The function takes an 'mne.Epochs' object as input and returns a numpy array of shape [n_samples, 5], where [n_samples] is the number of epochs in the input object.

The function first defines a dictionary, 'FREQ_BANDS', that specifies the frequency bands of interest, and then uses the 'psd_welch' function to compute the power spectral density (PSD) of the EEG signals in each epoch in these frequency bands.

The PSDs are then normalized and the mean PSD in each frequency band is computed. The resulting arrays are concatenated along the second axis to form the output numpy array.

# Classification 5-fold

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

In [ ]:
%%capture
features=[]
for d in range(len(data)):#get features from each epoch and save in a list
  features.append(eeg_power_band(data[d]))

In [ ]:
#convert list to array
features=np.concatenate(features)
features.shape

(7456, 84)

In [ ]:
#do 5 fold cross validation
clf=RandomForestClassifier()
accuracies=cross_val_score(clf, features,label,groups=group,cv=5)
print('Five fold accuracies',accuracies)
print('Average accuracy',np.mean(accuracies))

Five fold accuracies [0.71380697 0.69953052 0.63782696 0.73105298 0.69014085]
Average accuracy 0.6944716556712932


# Tips to improve the accuracy 
1. Try different classifiers like svm etc
2. Increasing number of training samples
3. Add some context to your data
4. Finetuning hyperparameters
5. Experiment with a different algorithm
6. Try feature elimination 
7. Add more variables and better feature precessing